# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = pd.read_csv ("weather_info.csv")
weather = weather.drop("Unnamed: 0", axis = 1)
weather

,Cities,Max Temp,Lat,Lng,Humidity,Wind Speed,Cloudiness,Country
0,pathein,84.15,16.78,94.73,77,8.81,2,MM
1,darnah,70.81,32.77,22.64,68,21.21,44,LY
2,kaitangata,52.00,-46.28,169.85,60,3.00,3,NZ
3,chokurdakh,20.75,70.63,147.92,96,5.03,82,RU
4,parrita,78.80,9.52,-84.32,65,8.05,40,CR
...,...,...,...,...,...,...,...,...
562,wilmington,78.01,34.23,-77.94,83,4.70,40,US
563,eydhafushi,85.01,5.10,73.07,75,15.61,100,MV
564,lae,77.81,-6.72,146.98,72,5.37,100,PG
565,la grande,42.01,45.32,-118.09,86,4.70,75,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

from ipywidgets.embed import embed_minimal_html
locations = weather[['Lat', 'Lng']]
weights=weather['Humidity']


# map types ['ROADMAP', 'HYBRID', 'TERRAIN', 'SATELLITE']
fig = gmaps.figure(map_type='TERRAIN', center=(0,0), zoom_level =3) 

heat_layer = gmaps.heatmap_layer(locations, weights=weights,max_intensity=0,point_radius = 30)

# Add layer
fig.add_layer(heat_layer)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

##### Create new DataFrame fitting weather criteria
* Make sure max temperature is not higher than 75 or lower than 65
* Ensure cloudiness is 0
* Wind is less than 10 mph

In [11]:
vacation_weather = weather.loc[(weather["Max Temp"]>70)&(weather["Max Temp"]<=80)]
vacation_weather = vacation_weather.loc[vacation_weather["Wind Speed"]<=10]
vacation_weather = vacation_weather.loc[vacation_weather["Cloudiness"]==0]
vacation_weather = vacation_weather.fillna("NM")
vacation_weather

,Cities,Max Temp,Lat,Lng,Humidity,Wind Speed,Cloudiness,Country
28,piranshahr,77.29,36.70,45.14,27,8.52,0,IR
29,turukhansk,74.86,65.82,87.98,28,9.46,0,RU
31,khakhea,75.38,-24.69,23.49,21,7.49,0,BW
194,changji,77.00,44.02,87.32,22,2.24,0,CN
301,sao joao da barra,78.80,-21.64,-41.05,65,6.93,0,BR
305,chizhou,70.30,30.66,117.48,81,4.25,0,CN
366,ritchie,75.20,-29.04,24.60,24,5.82,0,ZA
426,pahrump,72.00,36.21,-115.98,26,10.00,0,US
446,madzhalis,74.73,42.12,47.83,58,2.17,0,RU
509,dingtao,74.73,35.07,115.56,20,9.13,0,CN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = vacation_weather.copy()
hotel_df["Hotel Name"] = ''
hotel_df = hotel_df.rename(columns = {"Cities":"City"})
hotel_df

,City,Max Temp,Lat,Lng,Humidity,Wind Speed,Cloudiness,Country,Hotel Name
28,piranshahr,77.29,36.70,45.14,27,8.52,0,IR,
29,turukhansk,74.86,65.82,87.98,28,9.46,0,RU,
31,khakhea,75.38,-24.69,23.49,21,7.49,0,BW,
194,changji,77.00,44.02,87.32,22,2.24,0,CN,
301,sao joao da barra,78.80,-21.64,-41.05,65,6.93,0,BR,
305,chizhou,70.30,30.66,117.48,81,4.25,0,CN,
366,ritchie,75.20,-29.04,24.60,24,5.82,0,ZA,
426,pahrump,72.00,36.21,-115.98,26,10.00,0,US,
446,madzhalis,74.73,42.12,47.83,58,2.17,0,RU,
509,dingtao,74.73,35.07,115.56,20,9.13,0,CN,


In [13]:

target_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index,row in hotel_df.iterrows():
    city = row["City"]
    lat = row["Lat"]
    long = row["Lng"]
    params = {
        "location": f'{lat},{long}',
        "radius":5000,
        "keyword":"hotel",
        "key": g_key
    }


    response = requests.get(target_url, params).json()

    results = response['results']
    
    try:
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

        
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "No Hotel Found within 5000 m"
        


In [14]:
hotel_df
hotel_df = hotel_df.rename(columns = {"Cities": "City"})
hotel_df

,City,Max Temp,Lat,Lng,Humidity,Wind Speed,Cloudiness,Country,Hotel Name
28,piranshahr,77.29,36.70,45.14,27,8.52,0,IR,Hotel Darya
29,turukhansk,74.86,65.82,87.98,28,9.46,0,RU,"ООО ""Туруханская пушнина"""
31,khakhea,75.38,-24.69,23.49,21,7.49,0,BW,No Hotel Found within 5000 m
194,changji,77.00,44.02,87.32,22,2.24,0,CN,Garden Hotel
301,sao joao da barra,78.80,-21.64,-41.05,65,6.93,0,BR,Pousada Kactus
305,chizhou,70.30,30.66,117.48,81,4.25,0,CN,Chizhou Country Garden Phoenix Hotel
366,ritchie,75.20,-29.04,24.60,24,5.82,0,ZA,The Cygnature Boutique Hotel
426,pahrump,72.00,36.21,-115.98,26,10.00,0,US,Holiday Inn Express & Suites Pahrump
446,madzhalis,74.73,42.12,47.83,58,2.17,0,RU,No Hotel Found within 5000 m
509,dingtao,74.73,35.07,115.56,20,9.13,0,CN,Lavande Hotel


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

  
fig.add_layer(markers)
    
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))